In [15]:
import pandas as pd
import numpy as np
import seaborn as sns

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [17]:
df = sns.load_dataset('titanic')

In [18]:
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


In [33]:
df.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [40]:
df.select_dtypes(include=np.number).corr()

,survived,pclass,age,sibsp,parch,fare
survived,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307
pclass,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500
age,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566
sibsp,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651
parch,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225
fare,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000


In [19]:
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [20]:
df['age'].fillna(df['age'].mean(), inplace=True)

In [24]:
df['embarked'].fillna(df['embarked'].mode().values[0],inplace=True)
df['embark_town'].fillna(df['embark_town'].mode().values[0], inplace=True)

In [26]:
df.drop(columns=['deck'], inplace=True)

In [27]:
df.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [32]:
df_cat = df.select_dtypes(exclude=np.number)
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   sex          891 non-null    object  
 1   embarked     891 non-null    object  
 2   class        891 non-null    category
 3   who          891 non-null    object  
 4   adult_male   891 non-null    bool    
 5   embark_town  891 non-null    object  
 6   alive        891 non-null    object  
 7   alone        891 non-null    bool    
dtypes: bool(2), category(1), object(5)
memory usage: 37.7+ KB


In [45]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

for col in df_cat.columns:
    df_cat[col] = encoder.fit_transform(df_cat[col])
df[df_cat.columns] = df_cat

In [48]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, random_state = 42)

valid = test['survived']
test.drop(columns = ['survived'], inplace=True)

In [58]:
train_y = train['survived']
train_x=train.copy()
train_x.drop(columns=['survived'], inplace=True)

In [59]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(random_state=42)
model.fit(train_x,train_y)

pred = model.predict(test)

In [60]:
from sklearn.metrics import accuracy_score

print("예측 정확도 : ", accuracy_score(valid, pred))

예측 정확도 :  1.0


In [61]:
# 최소최대 척도(Min-Max Scale)로 변환 후 0.5보다 큰 값의 수 구하기

min_q = min(df['age'])
max_q = max(df['age'])

list_age = df['age']
list_fin = (list_age - min_q) / (max_q - min_q)

print(list_fin[list_fin>=0.5])
print(len(list_fin[list_fin>=0.5]))

6      0.673285
11     0.723549
15     0.685851
33     0.824076
35     0.522493
         ...   
862    0.597889
865    0.522493
871    0.585323
873    0.585323
879    0.698417
Name: age, Length: 150, dtype: float64
150


In [66]:
from sklearn.preprocessing import MinMaxScaler

data_q = pd.DataFrame(df['age'])


df_fin = MinMaxScaler().fit_transform(data_q)

print(df_fin[df_fin>=0.5])
print(len(df_fin[df_fin>=0.5]))

[0.67328474 0.72354863 0.68585072 0.8240764  0.52249309 0.61045489
 0.81151043 0.560191   0.57275697 0.7361146  0.88690626 0.58532295
 0.88062327 0.67328474 0.560191   0.58532295 0.52249309 0.63558683
 0.6921337  0.50364413 0.63558683 0.54762503 0.560191   0.76124654
 0.69841669 0.62302086 0.560191   0.54762503 0.72354863 0.52249309
 0.56647399 0.52249309 0.63558683 0.7361146  0.54762503 0.54762503
 0.67328474 0.77381252 0.50992712 0.62302086 0.6481528  0.72354863
 0.50992712 0.78637849 0.560191   0.81151043 0.52249309 0.62302086
 0.53505906 0.67328474 0.76124654 0.56647399 0.50992712 0.560191
 0.560191   0.52249309 0.560191   0.74868057 0.52249309 0.57275697
 0.63558683 0.54762503 0.52249309 0.62302086 0.79894446 0.560191
 0.6481528  0.61045489 0.81151043 0.62302086 0.59788892 0.58532295
 0.59788892 0.69841669 0.62302086 0.78637849 0.72354863 0.68585072
 0.88690626 0.67328474 0.67328474 0.58532295 0.54762503 0.50364413
 0.62302086 0.560191   0.62302086 0.79894446 0.77381252 0.59788892

In [67]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,1,22.0,1,0,7.2500,2,2,1,1,2,0,0
1,1,1,0,38.0,1,0,71.2833,0,0,2,0,0,1,0
2,1,3,0,26.0,0,0,7.9250,2,2,2,0,2,1,1
3,1,1,0,35.0,1,0,53.1000,2,0,2,0,2,1,0
4,0,3,1,35.0,0,0,8.0500,2,2,1,1,2,0,1


In [68]:
df['age'].replace(['나이'],inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_19468\1217125612.py:1: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df['age'].replace(['나이'],inplace=True)


In [71]:
df.rename(columns={'age':'나이'},inplace=True)

In [72]:
df

,survived,pclass,sex,나이,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,1,22.000000,1,0,7.2500,2,2,1,1,2,0,0
1,1,1,0,38.000000,1,0,71.2833,0,0,2,0,0,1,0
2,1,3,0,26.000000,0,0,7.9250,2,2,2,0,2,1,1
3,1,1,0,35.000000,1,0,53.1000,2,0,2,0,2,1,0
4,0,3,1,35.000000,0,0,8.0500,2,2,1,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0,13.0000,2,1,1,1,2,0,1
887,1,1,0,19.000000,0,0,30.0000,2,0,2,0,2,1,1
888,0,3,0,29.699118,1,2,23.4500,2,2,2,0,2,0,0
889,1,1,1,26.000000,0,0,30.0000,0,0,1,1,0,1,1


In [73]:
da = df['pclass']
da

0      3
1      1
2      3
3      1
4      3
      ..
886    2
887    1
888    3
889    1
890    3
Name: pclass, Length: 891, dtype: int64

In [74]:
dat = pd.DataFrame(da)

In [76]:
dat.columns = ['좌석']

In [77]:
dat

,좌석
0,3
1,1
2,3
3,1
4,3
...,...
886,2
887,1
888,3
889,1
